# Differentiation on Quantum Hardware

In [1]:
import pennylane as qml
from pennylane import numpy as np

np.set_printoptions(precision=10)  #printing a lot of floating point numbers

qnode is taking a Quantum function that has gates and measurement, then all of that is joined with a device either a simulator or a real hardware; so in that sense device will definitely come into play when we start taking derivatives of the Quantum function on the device. To take that into account when decorating the Quantum function with **@qml.qnode(dev)**, a keyword argument can also be specified called **diff_method** which specifies the differentiation method to be employed whenever you ask pennylane to take a gradient of the quantum function. <br>

You dont have to specify what diff_method is every time you make a qnode, you can leave it out what happens under the hood is pennylane chooses the most performant differentiation method thats compatible with the device.

In [2]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):   #params:angle of rotation
    qml.RY(params[0] ,wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1)) #expectation value of the linear combination of PauliZ operator on the first qubit and the second qubit

In [3]:
params = np.array([-np.pi/4, np.pi/4])
print(circuit(params))

1.414213562373095


**FINITE DIFFERENCE GRADIENT METHOD** (approximation to the actual gradient)<br>
calculating derivative by using a small but finite value instead of an infinitesimal value of h <br>
           $\frac{\delta f}{\delta x} \approx \frac{f(x+h)-f(x-h)}{2h}$;    $h<<1$

In [4]:
#our custom finite difference gradient function
def my_finite_diff_grad(params, h=1.0e-7):  #params is the parameters we wanna differentiate wrt
    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        params[i] += h
        gradient[i] += circuit(params)
        
        params[i] -= 2*h
        gradient[i] -= circuit(params)
        
        gradient[i] /= 2*h
        
        params[i] += h  #undo -h from params
        
    return gradient

@qml.qnode(dev, diff_method="finite-diff")
def circuit_finite_diff(params):
    qml.RY(params[0] ,wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [5]:
params = np.array([np.pi/4, np.pi/3], requires_grad = True)

print(my_finite_diff_grad(params))
print(qml.grad(circuit_finite_diff)(params))  #qml.grad: function which takes in qnode(circuit_finite_diff) and returns the gradient function of tht qnode

[-0.7071067798 -0.8660254025]
[-0.7071068131 -0.8660254314]


**PARAMETER SHIFT RULE** (partial derivatives of quantum circuit can be computed with linear combinations of circuit evaluations at shifted parameters)<br>
With some very reasonable assumptions on the gates in circuit, partial derivatives can be evaluated exactly with the formula <br>
$\frac{\delta f}{\delta \theta} = \sum_{\mu = 1}{2R}f(\theta + \frac{2\mu - 1}{2R}\pi) \frac{(-1)^{\mu - 1}}{4R \sin^{2}(\frac{2\mu - 1}{4R}\pi)}$  which simplies as following when the gates in circuit are Pauli-rotation gates <br>
$\frac{\delta f}{\delta \theta} = \frac{1}{2 \sin (s)}(f(\theta +s)-f(\theta -s))$, where s is the shift which doesnt have to be very small as in the finite difference differentiation method so we wont get lost in the noise

In [6]:
def my_parameter_shift_grad(params, s=np.pi/3):
    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        params[i] += s
        gradient[i] += circuit(params)
        
        params[i] -= 2*s
        gradient[i] -= circuit(params)
        
        gradient[i] /= 2*np.sin(s)
        
        params[i] += s  
        
    return gradient
     
@qml.qnode(dev, diff_method="parameter-shift")
def circuit_parameter_shift(params):
    qml.RY(params[0] ,wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [7]:
print(my_parameter_shift_grad(params))
print(qml.grad(circuit_parameter_shift)(params))

[-0.7071067812 -0.8660254038]
[-0.7071067812 -0.8660254038]
